# Gemini AloeChat

Demonstrates a simple multi-agent chatbot built with AloeGraph. You can download locally or run in Google Colab

**Quick Start:** Supply `GEMINI_API_KEY` as specifed in the Setup section and click `Run All`

## Setup

In [1]:
import aloegraph.V2.v2_aloe_graph as v2_aloe_graph
import aloegraph.V2.v2_base_response as v2_base_response
import aloegraph.V2.v2_base_model as v2_base_model
import aloegraph.V2.agent.v2_chat_model as v2_chat_model
import aloegraph.V2.agent.v2_intake_response as v2_intake_response
import aloegraph.V2.agent.v2_router_response as v2_router_response
import aloegraph.V2.agent.v2_intake_agent as v2_intake_agent
import aloegraph.V2.agent.v2_router_agent as v2_router_agent
import aloegraph.V2.agent.v2_chat_agent as v2_chat_agent
from aloegraph.dialog.notebook_chat_dialog import NotebookChatDialog

from aloegraph.V2.graph.v2_aloe_node import AloeNode, AloeEdge
from aloegraph.V2.response_generator import GeminiJSONResponseGenerator as Gemini
from aloegraph.V2.graph.v2_aloe_route import AloeRoute

from pydantic import Field
import json, time

import sys

GEMINI_API_KEY = None
if 'google.colab' in sys.modules:
  from google.colab import userdata
  # Download AloeGraph from GitHub if not running locally
  !pip install git+https://github.com/aloecraft/AloeGraph.git
  GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
  import os
  GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

from google import genai
client = genai.Client(api_key=GEMINI_API_KEY)

## 1. Define a state to retain for the duration of the chat

In this example,
- The top level state is a **`v2_chat_model.ChatAgentState`**
- The chat agent will only have visibility of **`ChatRouteState`**


In [2]:
class ChatRouteState(v2_base_model.V2AloeConfig):
    chat_history: list[str] = []
    chat_sequence_counter: int = 0

class SimpleChatState(v2_chat_model.ChatAgentState):
    chat_route_state: ChatRouteState = ChatRouteState()

## 2. Define an agent and response

In this example,

- The Gemini Model will respond with **`ChatRouteAgentResponse`**
    + extends **`v2_base_response.BaseResponse`**
    + contains a single field, **`agent_message`**
    + contains the system prompt, **`SYSTEM_INSTRUCTION`** which will accompany the user prompt

- **`ChatRouteAgent`** defines the agent graph
    + There is a single node, **`chatWithUser`**
    + Nodes are decorated with the **`@AloeNode`** decorator
    + The starting node is specified with `is_entry=True`
    + A node should specify at least one edge
    + Edges are decorated with the **`@AloeEdge`** decorator
    + A the target node for an edge is specified with `target=<Node Name>` or `target=v2_aloe_graph.END`

In [3]:

class ChatRouteAgentResponse(v2_base_response.BaseResponse):
    agent_message: str = Field(default_factory=str, description="Response to be shown to user")

    @classmethod
    def SYSTEM_INSTRUCTION(cls, agent_name: str, agent_description: str):
        return f"""
You are a friendly AI Chat Agent for {agent_name}.

**What is {agent_name}?**

> {agent_description}

**What To Know:**
- The current user message and chat history have been provided in the prompt
- Your task is to return a response to continue the conversation

**Output Description**
- Always return a JSON object with exactly this schema:
  {json.dumps(cls.model_json_schema(), indent=4)}
"""

class ChatRouteAgent(AloeRoute[SimpleChatState, ChatRouteState], v2_aloe_graph.V2AloeGraph[ChatRouteState]):
    def __init__(self, state: ChatRouteState, response_generator:Gemini[ChatRouteAgentResponse]):
        self.state = state
        self.response_generator:Gemini[ChatRouteAgentResponse] = response_generator

    @AloeNode(is_entry=True)
    @AloeEdge(target=v2_base_model.END)
    def chatWithUser(self, state: ChatRouteState) -> ChatRouteState:

        prompt=f"""
User Message:
---
{state.user_message}

Chat History:
---
{"\n".join([f" - {m}" for m in state.chat_history])}
"""
        
        self.notify_log(f"\n\n{prompt}\n\n")

        state.chat_history.append(f"{state.chat_sequence_counter} [USER - {time.strftime('%H:%M:%S')}] {state.user_message}")
        state.chat_sequence_counter += 1

        response = self.response_generator.generate(prompt)
        self.notify_log(f"Response:\n---\n{response}")

        state.chat_history.append(f"{state.chat_sequence_counter} [AGENT - {time.strftime('%H:%M:%S')}] {response.agent_message}")
        state.chat_sequence_counter += 1

        state.agent_message = response.agent_message
        return state

    def isAvailable(self, parent_state:SimpleChatState)->bool:
        return True
    
    def getRouteState(self, parent_state:SimpleChatState)->ChatRouteState:
        return parent_state.chat_route_state
    
    def setRouteState(self, parent_state:SimpleChatState, state: ChatRouteState) -> ChatRouteState:
        parent_state.chat_route_state = state
        self.state = state
        return state
    
    def describeRoute(self, parent_state:SimpleChatState)->str:
        return "Respond to user messages"


## 3. Create Chatbot

### First create an instance of the `ChatRouteAgent` we just defined

**You can use `generate_mermaid_chart` and `render_mermaid_chart` to view the agent graph**

e.g.
```
print(chat_route_agent.generate_mermaid_chart())
chat_route_agent.render_mermaid_chart()
```

In [4]:
AGENT_NAME = "Simple Graph Chat"
AGENT_DESCRIPTION = "This is simple chat application that has one functions: Respond to user messages"

simple_chat_state = SimpleChatState()
chat_response_generator = Gemini[ChatRouteAgentResponse](client, agent_name=AGENT_NAME, agent_description=AGENT_DESCRIPTION)
chat_route_agent = ChatRouteAgent(simple_chat_state.chat_route_state, chat_response_generator)

print(chat_route_agent.generate_mermaid_chart())
chat_route_agent.render_mermaid_chart()

graph
__START__([__START__]) --> chatWithUser[chatWithUser]
chatWithUser --> __END__([__END__])


### Next create an instance of `v2_chat_agent.ChatAgent`

- `v2_chat_agent.ChatAgent` allows you to specify `intake_agent` and `router_agent` to customize chat rules
- Add the newly created `chat_route_agent"` to `ChatAgent` by calling, `router_agent.addRoute(chat_route_agent, "chat_route_agent")`
- Set a reply notifier. `NotebookChatDialog` shows a chat dialog in a python notebook

In [5]:
intake_response_generator = Gemini[v2_intake_response.IntakeAgentResponse](client, agent_name=AGENT_NAME, agent_description=AGENT_DESCRIPTION)
router_response_generator = Gemini[v2_router_response.RouterAgentResponse](client, agent_name=AGENT_NAME, agent_description=AGENT_DESCRIPTION)

intake_agent = v2_intake_agent.IntakeAgent(simple_chat_state.intake_agent_state, intake_response_generator)
router_agent = v2_router_agent.RouterAgent(simple_chat_state.router_agent_state,router_response_generator)
router_agent.addRoute(chat_route_agent, "chat_route_agent")
simple_chat_agent = v2_chat_agent.ChatAgent(initial_state=simple_chat_state, intake_agent=intake_agent, router_agent=router_agent)

# Set a reply notifier. `NotebookChatDialog` shows a chat dialog in a python notebook
notebook_dialog = NotebookChatDialog()
simple_chat_agent.compile(reply_notifier=notebook_dialog, graph_name="simple_chat_agent")
print(simple_chat_agent.generate_mermaid_chart())
simple_chat_agent.render_mermaid_chart()

Graph compiled successfully.
Graph compiled successfully.
Graph compiled successfully.
Graph compiled successfully.
graph
__START__([__START__]) --> invoke_intake((invoke_intake))
invoke_route((invoke_route))
chat_message>chat_message]
auto_resume{{auto_resume}}
invoke_intake --> chat_message
invoke_intake --> invoke_route
invoke_route --> chat_message
chat_message --> auto_resume
auto_resume --> __END__([__END__])
auto_resume --> invoke_route


### Add the `simple_chat_agent` to `notebook_dialog` and call display to start chat

In [6]:
notebook_dialog.set_agent(simple_chat_agent)
notebook_dialog.display("Let's talk about peanuts")